In [1]:
import json
import ast
import re
import csv

In [2]:
# ---------------------------------------------------------
# 1. Tách các block JSON trong file TXT
# ---------------------------------------------------------
def extract_json_blocks(text: str):
    """
    Tìm tất cả các block dạng:
    ```json
    [...]
    ```
    hoặc
    ```python
    [...]
    ```
    hoặc standalone {...}
    """
    blocks = []

    pattern = r"```(?:json|python)?\s*(.*?)```"
    matches = re.findall(pattern, text, flags=re.DOTALL)

    for m in matches:
        candidate = m.strip()
        # Chỉ thêm block nếu có dấu [] hoặc {}
        if ("[" in candidate and "]" in candidate) or ("{" in candidate and "}" in candidate):
            blocks.append(candidate)

    # Nếu KHÔNG tìm thấy block code → fallback: toàn bộ text có thể là JSON
    if not blocks:
        cleaned = text.strip()
        if cleaned:
            blocks = [cleaned]

    return blocks



In [3]:
# ---------------------------------------------------------
# 2. Parse mỗi block thành list/dict Python
# ---------------------------------------------------------
def parse_block_to_python(block: str):
    block = block.strip()

    # Thử JSON trước
    try:
        return json.loads(block)
    except Exception:
        pass

    # Thử ast.literal_eval (cho data dạng Python dict)
    try:
        return ast.literal_eval(block)
    except Exception:
        pass

    # Thử trường hợp thiếu mảng ngoài
    try:
        wrapped = "[" + block + "]"
        return json.loads(wrapped)
    except Exception:
        pass

    print(block)

    raise ValueError("Không parse được block:\n" + block[:200])

In [4]:
# ---------------------------------------------------------
# 3. Chuẩn hóa một record (dict)
# ---------------------------------------------------------
def normalize_record(rec: dict):
    def to_list(v):
        if v is None:
            return []
        if isinstance(v, list):
            return v
        if isinstance(v, dict):
            return [v]
        return [v]

    # Map key tiếng Việt → key tiếng Anh
    mapping = {
        "Tên cây": "ten_cay",
        "tên cây": "ten_cay",
        "Loại bệnh": "loai_benh",
        "loại bệnh": "loai_benh",
        "Nguyên nhân": "nguyen_nhan",
        "nguyên nhân": "nguyen_nhan",
        "Triệu chứng": "trieu_chung",
        "triệu chứng": "trieu_chung",
        "Cách điều trị": "cach_dieu_tri",
        "cách điều trị": "cach_dieu_tri",
    }

    out = {
        "ten_cay": "",
        "loai_benh": "",
        "nguyen_nhan": [],
        "trieu_chung": [],
        "cach_dieu_tri": []
    }

    for k, v in rec.items():
        if k in mapping:
            mk = mapping[k]
            if mk in ["ten_cay", "loai_benh"]:
                out[mk] = v.strip() if isinstance(v, str) else str(v)
            else:
                out[mk] = to_list(v)
            # out[mk] = to_list(v)
        else:
            # key không map → bỏ qua
            pass

    return out

In [5]:
# ---------------------------------------------------------
# 4. Tạo pipeline TXT → JSON → CSV
# ---------------------------------------------------------
def pipeline(txt_path: str, json_path: str, csv_path: str):
    # Đọc file
    with open(txt_path, "r", encoding="utf-8") as f:
        raw = f.read()

    # Tách block JSON-like
    blocks = extract_json_blocks(raw)

    results = []

    for block in blocks:
        if block.startswith("Dưới đây") or block.startswith("Dựa trên nội dung") or block.startswith("Từ nội dung") or block.startswith("Dựa vào nội dung"):
            print(block)
            continue
        print("---------------")
        parsed = parse_block_to_python(block)
        # Trường hợp block là list
        if isinstance(parsed, list):
            for item in parsed:
                if isinstance(item, dict):
                    results.append(normalize_record(item))
        # Trường hợp block là dict
        elif isinstance(parsed, dict):
            results.append(normalize_record(parsed))
    # Ghi CSV
    with open(csv_path, "a", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["ten_cay", "loai_benh", "nguyen_nhan", "trieu_chung", "cach_dieu_tri"]
        )
        writer.writeheader()
        for r in results:
            writer.writerow({
                "ten_cay": r["ten_cay"],
                "loai_benh": r["loai_benh"],
                "nguyen_nhan": " | ".join(map(str, r["nguyen_nhan"])),
                "trieu_chung": " | ".join(map(str, r["trieu_chung"])),
                "cach_dieu_tri": " | ".join(map(str, r["cach_dieu_tri"])),
            })

    print(f"✔ Hoàn thành! Tổng số record: {len(results)}")
    print(f"→ JSON: {json_path}")
    print(f"→ CSV:  {csv_path}")

In [11]:
pipeline(
        txt_path="raw_data/raw_data_general_2.txt",
        json_path="clean_data_test1.json",
        csv_path="data/general_data_raw_3.csv"
    )


---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
---------------
--------